In [1]:
## This script will be used to create the confusion matrix
## as well as create some performance efficiency metrics

##  Acouple of notes I need to be aware of:
## 1. 628 is a nonsense code - I can remove these rows
## 2. I need to find a good way to protect against corrected answers
##   a. The stimuli are given a random interval any where between 1 and 1.5 seconds
##   in order to protect against this I need to find the windows between presentations
## 3. The goal is to determine how efficient these folks are at processing - so I will use the corrected answer
## it changes the answer 

In [2]:
#Stim Trigs (ID)
#1 (Crying)
#6 (Unhappy)
#7 (Neutral)
#8 (Happy)

#Response Trigs (ID)
#2 ("A")
#3 ("S")
#4 ("K")
#5 ("L")

#Stim Trigs (Care)
#3 (Crying)
#4 (Unhappy)
#5 (Neutral)
#6 (Happy)

#Response Trigs (Care)
#1 ("L Shift")
#2 ("R Shift")


In [3]:
import os
import pandas as pd
import numpy as np
from sklearn import metrics

In [10]:
## Run a test example here
df = pd.read_csv("/home/arosen/Documents/eegBehavioralData/ID/250-26#_Parent_Identify_ICAb.evt", sep='\t', header=0)

In [11]:
## First remove all 628 values from the df
df = df[-df["TriNo"].isin(["628"])]

In [12]:
## Now isolate the stimuli presentation from the answers
df_stim = df[df['TriNo'].isin(['1', '6', '7', '8'])]
df_stim.head(5)

,Tmu,Code,TriNo,Comnt
2,4867000,1,7,Trigger - DIN8
4,6882000,1,6,Trigger - DI64
7,10730000,1,7,Trigger - DIN8
9,12528000,1,1,Trigger - DI16
12,14460000,1,6,Trigger - DI64


In [6]:
%qtconsole

In [14]:
## Do the same for the reponses
df_resp = df[df['TriNo'].isin(['2', '3', '4', '5'])]
df_resp.head(5)

,Tmu,Code,TriNo,Comnt
3,5748000,1,2,Trigger - DI18
5,8881000,1,5,Trigger - DI32
8,11180000,1,3,Trigger - DI20
10,12980000,1,2,Trigger - DI18
11,13540000,1,4,Trigger - DI24


In [15]:
## Now I need to isolate the values that are inbetween intervals
## every stim should have 1 resp.. but there may be two for some
## This will be done in a for loop
## 118-314_OutResponse.csv 146

## and I will print the values that have multiple responses
## then I will have to take the second or first.... to be determined
row_vals = df_stim.shape[0]-1
## Now create an output data frame for everything to land into
d = pd.DataFrame([])
for x in range(row_vals):
    ## First identify the range
    lower_val = df_stim['Tmu         '].iloc[x]
    upper_val = df_stim['Tmu         '].iloc[x+1]
    ## Now check to see if the values are inbetween the interval in the resp
    if df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].shape[0] > 1:
        print("Case at " ,x)
        ## Grab the last response
        row_index = df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].shape[0] - 1
        #row_index = 0
        out_row = df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].iloc[row_index]
    
    elif df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].shape[0] == 0:
        print("Null response @ ", x)
        out_row = {'col1': [lower_val], 'col2': [np.nan], 'col3': [np.nan], 'col4': [np.nan]}
        out_row = pd.DataFrame(out_row)
        out_row.columns = df_stim.columns
    
    ## Now attach these all to the output
    else:
        out_row = df_resp[df_resp['Tmu         '].between(lower_val, upper_val)]
        
    input_stim = df_stim.iloc[x]
    d = d.append(input_stim)
    d = d.append(out_row)


Case at  3
Null response @  4
Null response @  7
Case at  10
Null response @  11
Case at  12
Null response @  15
Null response @  17
Case at  18
Null response @  19
Null response @  22
Null response @  23
Case at  24
Null response @  25
Case at  27
Case at  28
Case at  29
Case at  30
Null response @  31
Case at  36
Case at  38
Null response @  40
Case at  42
Null response @  43
Case at  44
Null response @  45
Null response @  48
Case at  49
Case at  50
Case at  52
Case at  53
Null response @  56
Null response @  57
Null response @  61
Null response @  62
Case at  65
Case at  68
Null response @  70
Null response @  71
Case at  73
Null response @  76
Null response @  77
Null response @  79
Case at  82
Null response @  83
Case at  84
Case at  90
Null response @  94
Null response @  95
Case at  96
Null response @  97
Case at  99
Case at  100
Case at  102
Null response @  105
Case at  108
Null response @  109
Case at  110
Null response @  111
Null response @  112
Null response @  113
Case a

In [22]:
## Now grab all of the even values - these are the stimuli
# We are going through this for two reasons:
# 1. AFGR is terrible at python programming
# 2. We want to get the differece in presentation and answer
#  to do 2 I am going to attach these togerther and calculate the differnece
df_stim = d[d['TriNo'].isin(['1', '6', '7', '8'])]
df_ans = d[-d['TriNo'].isin(['1', '6', '7', '8'])]
df_c = pd.concat([df_stim.reset_index(drop=True), df_ans.reset_index(drop=True)], axis=1)

In [23]:
## Now calculate the difference in time
df_c.columns = ['Index','CommentStim','TimeStim','TriNoStim','CodeAns','CommentAns', 'TimeAns', 'TriNoAns']

In [24]:
## Now create the difference between timeStim and TimeAns
df_c['timeToAnswer'] = df_c['TimeAns'] - df_c['TimeStim']

In [25]:
## Now create a confusion matrix
#print(metrics.classification_report(df_c['TriNoStim'], df_c['TriNoAns']))

TypeError: '<' not supported between instances of 'float' and 'str'

In [23]:
#!/usr/bin/env python
# coding: utf-8

# In[98]:


## This script will be used to create the confusion matrix
## as well as create some performance efficiency metrics

##  Acouple of notes I need to be aware of:
## 1. 628 is a nonsense code - I can remove these rows
## 2. I need to find a good way to protect against corrected answers
##   a. The stimuli are given a random interval any where between 1 and 1.5 seconds
##   in order to protect against this I need to find the windows between presentations
## 3. The goal is to determine how efficient these folks are at processing - so I will use the corrected answer
## it changes the answer 


# In[243]:


#Stim Trigs (ID)
#1 (Crying)
#6 (Unhappy)
#7 (Neutral)
#8 (Happy)

#Response Trigs (ID)
#2 ("A")
#3 ("S")
#4 ("K")
#5 ("L")

#Stim Trigs (Care)
#3 (Crying)
#4 (Unhappy)
#5 (Neutral)
#6 (Happy)

#Response Trigs (Care)
#1 ("L Shift")
#2 ("R Shift")


# In[244]:


import glob, os
import pandas as pd
import numpy as np
from sklearn import metrics


# In[245]:


## Run a test example here
df = pd.read_csv("/home/arosen/Documents/eegBehavioralData/ID/118-286_Parent_Identify-export.evt", sep='\t', header=0)
os.chdir("/home/arosen/Documents/eegBehavioralData/ID/")
for df in glob.glob("*.evt"):
    print(df)
    ## Now create the output name while we have the string in memory
    out_name = "/home/arosen/Documents/eegBehavioralData/ID_mod/" + df.replace("#", "").partition("_")[0] + "_OutResponse.csv"
    df = pd.read_csv(df, sep='\t', header=0)

    
    
    ## First remove all 628 values from the df
    df = df[-df["TriNo"].isin(["628"])]
    
    
    # In[247]:
    
    
    ## Now isolate the stimuli presentation from the answers
    df_stim = df[df['TriNo'].isin(['1', '6', '7', '8'])]
    df_stim.head(5)
    
    
    # In[248]:
    
    
    ## Do the same for the reponses
    df_resp = df[-df['TriNo'].isin(['1', '6', '7', '8'])]
    df_resp.head(5)
    
    
    # In[249]:
    
    
    ## Now I need to isolate the values that are inbetween intervals
    ## every stim should have 1 resp.. but there may be two for some
    ## This will be done in a for loop
    ## and I will print the values that have multiple responses
    ## then I will have to take the second or first.... to be determined
    row_vals = df_stim.shape[0]-1
    ## Now create an output data frame for everything to land into
    d = pd.DataFrame([])
    for x in range(row_vals):
        ## First identify the range
        lower_val = df_stim['Tmu         '].iloc[x]
        upper_val = df_stim['Tmu         '].iloc[x+1]
        ## Now check to see if the values are inbetween the interval in the resp
        if df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].shape[0] > 1:
            print("Case at " ,x)
            ## Grab the last response
            row_index = df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].shape[0] - 1
            #row_index = 0
            out_row = df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].iloc[row_index]
        
        elif df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].shape[0] == 0:
            print("Null response @ ", x)
            out_row = {'col1': [lower_val], 'col2': [np.nan], 'col3': [np.nan], 'col4': [np.nan]}
            out_row = pd.DataFrame(out_row)
            out_row.columns = df_stim.columns
        
        ## Now attach these all to the output
        else:
            out_row = df_resp[df_resp['Tmu         '].between(lower_val, upper_val)]
            
        input_stim = df_stim.iloc[x]
        input_stim['numOfResponses'] = df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].shape[0]
        ## Now add a column with the number of responses
        d = d.append(input_stim)
        d = d.append(out_row)


    ## Now grab all of the even values - these are the stimuli
    # We are going through this for two reasons:
    # 1. AFGR is terrible at python programming
    # 2. We want to get the differece in presentation and answer
    #  to do 2 I am going to attach these togerther and calculate the differnece
    df_stim = d[d['TriNo'].isin(['1', '6', '7', '8'])]
    df_ans = d[-d['TriNo'].isin(['1', '6', '7', '8'])]
    df_c = pd.concat([df_stim.reset_index(drop=True), df_ans.reset_index(drop=True)], axis=1)


    ## Now calculate the difference in time
    df_c.columns = ['Index','CommentStim','TimeStim','TriNoStim','noOfResposesStim','CodeAns','CommentAns', 'TimeAns', 'TriNoAns', 'numOfResponsesAns']



    ## Now create the difference between timeStim and TimeAns
    df_c['timeToAnswer'] = df_c['TimeAns'] - df_c['TimeStim']

    ## Now write the output
    df_c.to_csv(out_name)








222-13#_Parent_Identify.evt
Null response @  1
Case at  6
Null response @  7
Case at  8
Case at  12
Case at  14
Null response @  15
Null response @  19
Null response @  23
Case at  24


/home/arosen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/arosen/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Case at  40
Case at  54
Case at  62
Case at  66
Case at  80
Case at  85
Case at  106
Null response @  115
Case at  120
Case at  155
Null response @  171
Case at  182
Null response @  186
Case at  201
Case at  234
Case at  269
Case at  288
Case at  302
Case at  347
250-23#_Parent_Identify_ICAb.evt
Null response @  11
Null response @  111
Case at  112
Null response @  255
Null response @  280
Null response @  326
Case at  367
118-229_Parent_Identify-export.evt
Case at  104
Null response @  182
Null response @  186
Null response @  191
Case at  288
Case at  342
Case at  361
Case at  368
118-107_Parent_Identify-export.evt
Null response @  98
Case at  100
Case at  145
Null response @  172
Case at  173
Case at  180
Case at  200
Case at  214
Case at  229
Case at  234
Case at  260
Case at  273
Null response @  313
Case at  314
Null response @  319
Case at  320
Null response @  325
Case at  331
Case at  334
Case at  366
117-76_Parent_Identify-export.evt
Null response @  14
Null response @  61
N

Null response @  176
Null response @  178
Case at  180
Null response @  181
Case at  182
Null response @  184
Null response @  185
Case at  186
Null response @  189
Case at  190
Null response @  191
Case at  193
Case at  194
Null response @  195
Case at  196
Case at  197
Null response @  198
Null response @  200
Null response @  201
Null response @  206
Null response @  208
Case at  209
Null response @  210
Case at  212
Null response @  214
Case at  215
Case at  217
Case at  219
Null response @  220
Null response @  224
Null response @  225
Case at  226
Null response @  227
Case at  228
Case at  234
Null response @  239
Case at  240
Null response @  241
Case at  243
Case at  244
Case at  245
Case at  246
Null response @  247
Null response @  248
Case at  252
Case at  254
Case at  258
Case at  260
Null response @  261
Case at  265
Case at  266
Null response @  267
Case at  268
Case at  269
Null response @  273
Null response @  280
Case at  281
Case at  283
Case at  284
Null response @  

222-16_Parent_Identify-export.evt
Null response @  4
Null response @  5
Null response @  6
Case at  20
Null response @  38
Null response @  226
Case at  241
Null response @  269
Null response @  346
Null response @  353
119-102_Parent_Identify-export.evt
Case at  166
Case at  220
Case at  308
Null response @  339
Null response @  342
Null response @  345
Null response @  347
118-62#_Parent_Identify-export.evt
Null response @  4
Null response @  5
Null response @  6
Case at  20
Null response @  38
Null response @  226
Case at  241
Null response @  269
Null response @  346
Null response @  353
118-172_Parent_Identify-export.evt
Null response @  9
Case at  28
Null response @  77
Case at  112
Case at  236
Case at  245
117-46#_Parent_Identify-export.evt
Null response @  0
Null response @  1
Null response @  17
Null response @  18
Null response @  21
Null response @  36
Null response @  39
Null response @  40
Null response @  45
Null response @  72
Null response @  82
Null response @  122
Nu

Case at  106
Null response @  114
Case at  142
Case at  143
Case at  145
Null response @  147
Null response @  154
Null response @  155
Null response @  156
Null response @  157
Null response @  159
Null response @  163
Null response @  165
Null response @  169
Null response @  170
Null response @  179
Null response @  185
Null response @  187
Null response @  190
Null response @  191
Null response @  192
Case at  194
Null response @  195
Null response @  196
Null response @  198
Null response @  204
Null response @  205
Null response @  239
Null response @  240
Case at  241
Null response @  242
Null response @  258
Null response @  265
Null response @  267
Null response @  268
Null response @  269
Null response @  271
Case at  272
Case at  280
Case at  284
Null response @  285
Null response @  293
Case at  301
Null response @  330
Null response @  334
Null response @  335
Case at  342
Case at  343
121-4##_Parent_Identify-export.evt
Null response @  1
Null response @  2
Null response @

In [ ]:
## Now I want to find all of t